In [10]:
from agents import (
    Agent,
    WebSearchTool,
    trace,
    Runner,
    gen_trace_id,
    function_tool,
    AsyncOpenAI,
)
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [11]:
load_dotenv(override=True)

True

In [12]:
import os

from agents import OpenAIChatCompletionsModel

GEMINI_BASE_URL = os.getenv(
    "GEMINI_BASE_URL", "https://generativelanguage.googleapis.com/v1beta/openai/"
)

gemini_client = AsyncOpenAI(
    base_url=GEMINI_BASE_URL, api_key=os.getenv("GEMINI_API_KEY")
)

gemini_model = OpenAIChatCompletionsModel(
    openai_client=gemini_client, model="gemini-2.0-flash"
)

In [13]:
from tavily import TavilyClient
tavily_api_key = os.getenv("TAVILY_API_KEY")

@function_tool()
def search_the_internet(query:str):
    """
    Search the internet for the given query.
    """
    tavily_client = TavilyClient(api_key=tavily_api_key)
    response = tavily_client.search(query)
    return response

In [14]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[search_the_internet],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [15]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

AI agent frameworks in 2025 include LangChain, AgentFlow, AutoGen, Semantic Kernel, Atomic Agents, CrewAI, and RASA. Also mentioned are Microsoft AutoGen, Phidata, PromptFlow, and OpenAI Swarm. These frameworks provide building blocks for developing, deploying, and managing AI agents, offering built-in features and functions.

Other frameworks mentioned: LangGraph, OpenAI Agents SDK (aka Swarm), SuperAGI, IBM watsonx. These frameworks help businesses drive innovation and growth.
